# AML Project - Abstractive Text Summarization

Problem Statment: Given a news article, generate a summary of two-to-three sentences and a headline for the article.

The summary should be abstractive rather than extractive. In abstractive summarization, new sentences are generated as part of the summary and the sentences in the summary might not be present in the news article.

### Importing necessary packages

In [48]:
import numpy as np
import pandas as pd

# splittin dataset
from sklearn.model_selection import train_test_split

# evaluation metric
from ignite.metrics import Rouge, RougeN, RougeL

# visualisation
# import plotly.express as px
# import plotly.graph_objects as go
# import plotly.offline as pyo
 
# pyo.init_notebook_mode() 

### Importing the Indian News Summary Dataset

In [49]:
df_headline = pd.read_csv('dataset/news_headline.csv', header=0)
df_headline

,text,summary
0,"Saurav Kant, an alumnus of upGrad and IIIT-B's...",upGrad learner switches to career in ML & Al w...
1,Kunal Shah's credit card bill payment platform...,Delhi techie wins free food from Swiggy for on...
2,New Zealand defeated India by 8 wickets in the...,New Zealand end Rohit Sharma-led India's 12-ma...
3,"With Aegon Life iTerm Insurance plan, customer...",Aegon life iTerm insurance plan helps customer...
4,Speaking about the sexual harassment allegatio...,"Have known Hirani for yrs, what if MeToo claim..."
...,...,...
98396,A CRPF jawan was on Tuesday axed to death with...,CRPF jawan axed to death by Maoists in Chhatti...
98397,"'Uff Yeh', the first song from the Sonakshi Si...",First song from Sonakshi Sinha's 'Noor' titled...
98398,"According to reports, a new version of the 199...",'The Matrix' film to get a reboot: Reports
98399,A new music video shows rapper Snoop Dogg aimi...,Snoop Dogg aims gun at clown dressed as Trump ...


In [50]:
print(df_headline.shape)

(98401, 2)


### Splitting the dataset in training and testing set (80-20) split

In [27]:
x_train, x_test, y_train, y_test = train_test_split(df_headline['text'], df_headline['summary'], test_size=0.2,random_state=25, shuffle=True)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

x_train_list, y_train_list = x_train.tolist(), y_train.tolist()
x_test_list, y_test_list = x_test.tolist(), y_test.tolist()

print(len(x_train_list), len(y_train_list))
print(len(x_test_list), len(y_test_list))

(78720,) (78720,)
(19681,) (19681,)
78720 78720
19681 19681


### Importing the Pegasus model, tokenizer, trainer, and training arguments for finetuning

In [51]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizerFast, Trainer, TrainingArguments
import torch
print(torch.__version__)
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)

1.10.2
cpu


In [29]:
# %%time
# tokenizer_cnn = PegasusTokenizerFast.from_pretrained("google/pegasus-cnn_dailymail")
# model_cnn = PegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail").to(torch_device)

CPU times: user 12.5 s, sys: 7.95 s, total: 20.5 s
Wall time: 25.5 s


In [52]:
%%time
tokenizer_large = PegasusTokenizerFast.from_pretrained("google/pegasus-large")
model_large = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large").to(torch_device)

CPU times: user 12.1 s, sys: 8.25 s, total: 20.4 s
Wall time: 29.7 s


In [53]:
# configuaration of tokenizer
tokenizer_large

PreTrainedTokenizerFast(name_or_path='google/pegasus-large', vocab_size=96103, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', '<u

In [110]:
# parameters of the model
# model_large

In [56]:
# number of trainable parameters
model_large_params = sum(p.numel() for p in model_large.parameters() if p.requires_grad)
print(model_large_params)

568699904


### Testing the pretrained Pegasus model on our test data

In [57]:
# function to get summary of a text of list of texts
def get_summary(tokenizer, model, x):
    x_tokenized = tokenizer(x, truncation=True, padding = True, return_tensors="pt").to(torch_device)
    print("Input X tokenized. Generating Summary ...")
    y_pred_tokenized= model.generate(**x_tokenized).to(torch_device)
    print("Summary Generated. Decoding Summary ...")
    y_pred = tokenizer.batch_decode(y_pred_tokenized, skip_special_tokens=True)
    print("Summary Decoded.")
    return y_pred

# function to caluculate rogue score
def calculate_rouge(m, y_pred, y):
    candidate = [i.split() for i in y_pred]
    reference = [i.split() for i in y]
    # print(candidate, reference)
    
    m.update((candidate, reference))
    
    return m.compute()

In [42]:
%%time
m = Rouge(variants=["L", 1], multiref="best")

for i in range(0,25, 25):
    y_test_pred = get_summary(tokenizer_large,model_large, x_test_list[i:i+25])
    r = calculate_rouge(m, y_test_pred, y_test_list[i:i+25])
    print(r)

Ignored unknown kwarg option direction
Input X tokenized. Generating Summary ...
Summary Generated. Decoding Summary ...
Summary Decoded.
{'Rouge-L-P': 0.027428257223599803, 'Rouge-L-R': 0.13904761904761906, 'Rouge-L-F': 0.13904761904761906, 'Rouge-1-P': 0.027428257223599803, 'Rouge-1-R': 0.13904761904761906, 'Rouge-1-F': 0.13904761904761906}
CPU times: user 52min 21s, sys: 53min 50s, total: 1h 46min 12s
Wall time: 28min 30s


In [58]:
x_test_list[:10]

["The world's largest cruise ship, which is 1,188 feet long and weighs 228,000 tonnes, has been delivered to cruise line Royal Caribbean ahead of its maiden voyage. The 'Symphony of the Seas' ship features a zipline and an ice rink. It has over 2,700 guest rooms, including two-storey family suites that have a slide connecting the second and first floors.",
 'Debt-ridden Air India has received a loan worth â\x82¹1,500 crore from Bank of India to meet urgent working capital needs, according to reports. This comes less than a month after Air India floated a tender in this regard. Prior to this, the airline had reportedly borrowed around â\x82¹3,250 crore as short-tenure loans from IndusInd Bank and Punjab National Bank.',
 'China-based scientists have discovered a method to convert tree leaves into a porous carbon material that can be used to make electronics. Dried phoenix tree leaves, when grounded and heated to 220 ÂºC, produced a powder of carbon microspheres. On further heating with 

In [61]:
y_test_pred[:10]

["The 'Symphony of the Seas' ship features a zipline and an ice rink.",
 'Debt-ridden Air India has received a loan worth 11,500 crore from Bank of India to meet urgent working capital needs, according to reports.',
 'Dried phoenix tree leaves, when grounded and heated to 220 oC, produced a powder of carbon microspheres.',
 'Novogratz said, "We didn\'t like market conditions and we wanted to re-evaluate what we\'re doing." He further added, "I didn\'t want to have to deal with the schizophrenic emotional side of it."',
 'Juicero had said its 126,000 juicer applies four tons of force to the packs to squeeze the juice.',
 '"We would like to know what kind of powers the RBI Governor needs," a panel official said.',
 'It has allowed exchanges to set trading hours in equity derivatives between 9:00 am and 11:55 pm, effective October 1, 2018.',
 'The film will revolve around the story of a lonely middle-aged spinster, portrayed by Manisha.',
 'Speaking about being trolled for her dialogue, "

In [62]:
y_test_list[:10]

["World's largest cruise ship with zipline, ice rink delivered",
 'Air India gets â\x82¹1,500-crore loan from Bank of India',
 'Scientists convert dried tree leaves into electronics',
 'Investor halts plan for $500 mn cryptocurrency hedge fund',
 'Juicer startup refunds users on hand-squeezing fruit packs',
 'Parliamentary panel summons RBI Governor over bank scams',
 "Exchanges allowed to extend equity derivatives' trading time",
 "Trailer of Manisha Koirala's comeback film 'Dear Maya' out",
 "I've become more popular since 'Race 3' memes: Daisy Shah",
 "Former B'desh PM Khaleda Zia jailed for 7 years in corruption case"]

### Preparing our training dataset for finetuning our model

In [77]:
# dataset class to efficiently manage our dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, text, summary):
        self.text= text
        self.summary = summary
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.text.items()}
        item['labels'] = torch.tensor(self.summary['input_ids'][idx])  # torch.tensor(self.summary[idx])
        return item
    def __len__(self):
        return len(self.text['input_ids'])

# function to prepare training data for model fine-tuning
def prepare_data(tokenizer, x_train, y_train, x_val=None, y_val=None):
    
    val = False if x_val is None or y_val is None else True
    def tokenize_data(text, summary):
        text_tokenized = tokenizer(text, truncation=True, padding=True)
        summary_tokenized = tokenizer(summary, truncation=True, padding=True)
        dataset_tokenized = Dataset(text_tokenized, summary_tokenized)
        return dataset_tokenized
    
    train_dataset = tokenize_data(x_train, y_train)
    val_dataset = tokenize_data (x_val, y_val) if val else None
    
    return train_dataset, val_dataset

In [78]:
# function to prepare and configure base model for fine-tuning
def prepare_finetuning(model, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):

  if freeze_encoder:                                # if freeze_encoder is true
    for param in model.model.encoder.parameters():  # freeze the encode parameters
      param.requires_grad = False

  if val_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,            # output directory
      adafactor=True,                   # use adafactor instead of AdamW
      num_train_epochs=10,              # total number of training epochs
      per_device_train_batch_size=8,    # batch size per device during training
      per_device_eval_batch_size=8,     # batch size for evaluation
      save_steps=500,                   # number of updates steps before checkpoint saves
      save_total_limit=5,               # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',      # evaluation strategy to adopt during training
      eval_steps=100,                   # number of update steps before evaluation
      warmup_steps=500,                 # number of warmup steps for learning rate scheduler
      weight_decay=0.01,                # strength of weight decay
      logging_dir='./logs',             # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated transformer model
      args=training_args,                  # training arguments as defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset             # evaluation dataset
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      adafactor=True,                  # use adafactor instead of AdamW
      num_train_epochs=5,             # total number of training epochs
      per_device_train_batch_size=10,   # batch size per device during training, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=50,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                     # the instantiated transformer model
      args=training_args,              # training arguments as defined above
      train_dataset=train_dataset,     # training dataset
    )

  return trainer

In [116]:
# tokenizing the training dataset
train_dataset,_ = prepare_data(tokenizer_cnn, x_train_list[:10000], y_train_list[:10000])

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


In [121]:
train_dataset[10]

{'input_ids': tensor([27652, 12118,  3670,  2965, 37735, 93740,   124,  1491,   196,  1921,
           122,   114, 76400,   328,   134,   153,   480,   115, 91623, 31306,
           107,  4979,   273,  4851,   244,   988,   109, 13093,   108,   109,
         76400,   729,   243,   108,   198,   284,   394,   666,   120,   178,
           143, 12066, 37735,   158,   192,   331,   111,   133,   425,   122,
           214,   496, 13093, 37735,   243,   109,   696,   140,   142,  2353,
           112,  1111,   165,   112,  1074,   755,   200,   107,     1,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [80]:
%%time

trainer = prepare_finetuning(model_large, train_dataset) # compile the trainer model

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


CPU times: user 16 ms, sys: 39 ms, total: 55 ms
Wall time: 119 ms


In [70]:
# %%time
# trainer.train()

## Model was finetuned on the HPC

In [135]:
x_test_list[:10]

["The world's largest cruise ship, which is 1,188 feet long and weighs 228,000 tonnes, has been delivered to cruise line Royal Caribbean ahead of its maiden voyage. The 'Symphony of the Seas' ship features a zipline and an ice rink. It has over 2,700 guest rooms, including two-storey family suites that have a slide connecting the second and first floors.",
 'Debt-ridden Air India has received a loan worth â\x82¹1,500 crore from Bank of India to meet urgent working capital needs, according to reports. This comes less than a month after Air India floated a tender in this regard. Prior to this, the airline had reportedly borrowed around â\x82¹3,250 crore as short-tenure loans from IndusInd Bank and Punjab National Bank.',
 'China-based scientists have discovered a method to convert tree leaves into a porous carbon material that can be used to make electronics. Dried phoenix tree leaves, when grounded and heated to 220 ÂºC, produced a powder of carbon microspheres. On further heating with 

In [104]:
# base model prediction
y_test_pred[:10]

["The 'Symphony of the Seas' ship features a zipline and an ice rink.",
 'Debt-ridden Air India has received a loan worth 11,500 crore from Bank of India to meet urgent working capital needs, according to reports.',
 'Dried phoenix tree leaves, when grounded and heated to 220 oC, produced a powder of carbon microspheres.',
 'Novogratz said, "We didn\'t like market conditions and we wanted to re-evaluate what we\'re doing." He further added, "I didn\'t want to have to deal with the schizophrenic emotional side of it."',
 'Juicero had said its 126,000 juicer applies four tons of force to the packs to squeeze the juice.',
 '"We would like to know what kind of powers the RBI Governor needs," a panel official said.',
 'It has allowed exchanges to set trading hours in equity derivatives between 9:00 am and 11:55 pm, effective October 1, 2018.',
 'The film will revolve around the story of a lonely middle-aged spinster, portrayed by Manisha.',
 'Speaking about being trolled for her dialogue, "

In [105]:
y_test_list[:10]

["World's largest cruise ship with zipline, ice rink delivered",
 'Air India gets â\x82¹1,500-crore loan from Bank of India',
 'Scientists convert dried tree leaves into electronics',
 'Investor halts plan for $500 mn cryptocurrency hedge fund',
 'Juicer startup refunds users on hand-squeezing fruit packs',
 'Parliamentary panel summons RBI Governor over bank scams',
 "Exchanges allowed to extend equity derivatives' trading time",
 "Trailer of Manisha Koirala's comeback film 'Dear Maya' out",
 "I've become more popular since 'Race 3' memes: Daisy Shah",
 "Former B'desh PM Khaleda Zia jailed for 7 years in corruption case"]

In [134]:
with open('train.log', 'r') as f:
    text = f.read()
    
print(text)

Torch Version: 1.11.0+cu102
Available Torc Device: cuda

Reading the Dataset...
(98401, 2)

Splitting the Dataset...
(78720,) (78720,)
(19681,) (19681,)
Length of the Training and Test Set...
78720 78720
19681 19681

Tokening the dataset
Length of the dataset: 10000

Preparing model_large for finetuning...

Training...
{'loss': 9.2984, 'learning_rate': 2.5e-06, 'epoch': 0.02}
{'loss': 9.1668, 'learning_rate': 5e-06, 'epoch': 0.04}
{'loss': 9.0949, 'learning_rate': 7.5e-06, 'epoch': 0.06}
{'loss': 8.6245, 'learning_rate': 1e-05, 'epoch': 0.08}
{'loss': 8.0859, 'learning_rate': 1.25e-05, 'epoch': 0.1}
{'loss': 7.6655, 'learning_rate': 1.5e-05, 'epoch': 0.12}
{'loss': 7.4451, 'learning_rate': 1.75e-05, 'epoch': 0.14}
{'loss': 6.9593, 'learning_rate': 2e-05, 'epoch': 0.16}
{'loss': 7.0623, 'learning_rate': 2.25e-05, 'epoch': 0.18}
{'loss': 6.6862, 'learning_rate': 2.5e-05, 'epoch': 0.2}
{'loss': 6.6415, 'learning_rate': 2.7500000000000004e-05, 'epoch': 0.22}
{'loss': 6.2933, 'learning_rate

### Loading finetuned model from checkpoint

In [147]:
model_path = "./results/checkpoint-5000/"
model_finetune = PegasusForConditionalGeneration.from_pretrained(model_path, local_files_only=True).to(torch_device)

loading configuration file ./results/checkpoint-5000/config.json
Model config PegasusConfig {
  "_name_or_path": "google/pegasus-large",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": 

In [149]:
test_text = """
Elon Musk said on Tuesday that he would “reverse the permanent ban” of former President Donald J. Trump on Twitter and let him back on the social network, in one of the first specific comments by Mr. Musk, the world’s richest man, of how he would change the social media service.

Mr. Musk, who struck a deal last month to buy Twitter for $44 billion, said at a Financial Times conference that the company’s decision to bar Mr. Trump last year for tweets about the riots at the U.S. Capitol was “a mistake because it alienated a large part of the country and did not ultimately result in Donald Trump not having a voice.” He added that it was “morally wrong and flat-out stupid” and that “permanent bans just fundamentally undermine trust in Twitter.”

Mr. Musk’s remarks were a preview of the kinds of sweeping changes he might make at Twitter, which he is expected to take ownership of in the next six months. The billionaire, who also leads the electric carmaker Tesla and the rocket company SpaceX, has called himself a “free speech absolutist” and has said he is unhappy with how Twitter decides what can and cannot be posted online.

But up until Tuesday, Mr. Musk, 50, had spoken mostly in general terms and had not singled out Twitter accounts that might be affected by his takeover. He had called free speech “the bedrock of a functioning democracy” and had spoken of his desire to give people more control over their own social media feeds. By specifying that Mr. Trump could return to the platform, Mr. Musk uncorked a political firestorm.

Mr. Trump wielded Twitter for many years as both a megaphone and a cudgel, rallying his 88 million followers on issues such as immigration while also going after opponents. That avenue was cut off in January 2021 when Twitter, along with Facebook and other platforms, barred Mr. Trump from posting in the wake of the attack on the U.S. Capitol building. Twitter said at the time that Mr. Trump had violated policies and risked inciting violence among his supporters. Facebook barred Mr. Trump for similar reasons.

Mr. Trump, who has since begun a social media site called Truth Social, did not respond to a request for comment. Last month, Mr. Trump said that even with Mr. Musk buying Twitter, he did not plan to return to the platform and was “going to stay on Truth.”

Twitter declined to comment.

Derrick Johnson, the president of the N.A.A.C.P., said that free speech online needed to come with guardrails.

“Mr. Musk: Free speech is wonderful, hate speech is unacceptable,” he said. “Do not allow 45 to return to the platform. Do not allow Twitter to become a petri dish for hate speech or falsehoods that subvert our democracy.”

But Jack Dorsey, a founder and board member of Twitter, tweeted that permanent suspensions of individual users “are a failure” of the company and largely “don’t work.” Mr. Dorsey, who was chief executive of Twitter when Mr. Trump was barred, had said last year that booting the president was the right decision for the company, but backtracked on Tuesday by calling it “a business decision” and saying “it shouldn’t have been.”
Editors’ Picks
The Magazine Business, From the Coolest Place to the Coldest One
Farewell to the iPod
Francis Fukuyama Predicted the End of History. It’s Back (Again).

Even with Mr. Musk’s comments, Mr. Trump’s potential return to Twitter remains far from assured. Mr. Musk is mercurial and has a history of saying things he does not follow through on. In 2018, he famously declared that he planned to take Tesla private and had the funding to do so, when he did not. Even his most devoted followers sometimes wonder whether his obscurantist tweets are serious or are made in jest.

Investors have also questioned whether Mr. Musk’s deal for Twitter will be completed. The company’s shares closed Tuesday at $47.26, well below the $54.20 that Mr. Musk agreed to pay for them. Mr. Musk is also still securing financing for his takeover. While venture-capital firms and some big banks have lined up to invest, the billionaire is on the hook to provide as much as $21 billion of his own cash. He has not detailed where he will obtain that money.

Mr. Musk referred to the possibility of the deal not closing on Tuesday and of nothing happening with Mr. Trump’s Twitter account. “Obviously, I don’t own Twitter yet,” he said at the Financial Times conference. “So this is not a thing that will definitely happen because what if I don’t own Twitter.”

Mr. Trump long bedeviled social media companies because of how he pushed the line on speech, sometimes spreading lies and bullying people. But the companies’ moves to bar him drew accusations, especially from conservatives, that they were engaging in censorship and were biased against Republican voices.

Mr. Musk seemed to echo some of those conservative complaints on Tuesday, accusing Twitter of “a strong left bias, because it’s based in San Francisco” and saying “victory would be that the most far-right 10 percent and the most far-left 10 percent are equally upset.”

Some of the companies have since shied from appearing as the final word on who gets to say what online. Facebook referred Mr. Trump’s case to its Oversight Board, a company-appointed panel of academics, journalists and former members of government. The board ruled that Facebook was right to bar Mr. Trump, but it said the company had not thoroughly explained its decision and should revisit an indefinite suspension.

In June, Facebook said Mr. Trump’s ban would last at least two years, keeping the former president off the site through the 2022 midterm elections.

When Mr. Musk began buying up shares of Twitter this year, he started voicing more of his thoughts about the service and free speech, including in exchanges with Mr. Dorsey. In March, Mr. Musk asked his followers if Twitter was failing to adhere to free speech principles.
How Elon Musk Bought Twitter
Card 1 of 6

A blockbuster deal. Elon Musk, the world’s wealthiest man, capped what seemed an improbable attempt by the famously mercurial billionaire to buy Twitter for roughly $44 billion. Here’s how the deal unfolded:

The initial offer. Mr. Musk made an unsolicited bid worth more than $40 billion for the influential social network, saying that he wanted to make Twitter a private company and that he wanted people to be able to speak more freely on the service.

The response. Twitter’s board countered Mr. Musk’s offer with a defense mechanism known as a “poison pill.” This well-worn corporate tactic makes a company less palatable to a potential acquirer by making it more expensive for them to buy shares above a certain threshold.

Securing financing. Though his original offer had scant details and was received skeptically by Wall Street, Mr. Musk has been moving swiftly to secure commitments worth $46.5 billion to finance his bid, putting pressure on Twitter’s board to take his advances seriously.

Striking a deal. With the financing in place, Twitter’s board met with Mr. Musk to discuss his offer. The two sides soon reached a deal, with the social media company agreeing to sell itself for $54.20 a share.

What’s next? Shareholders will vote on the offer, which will also be reviewed by regulators. The deal is expected to take three to six months to close. In the meantime, scrutiny is likely to be intense and several questions remain about Mr. Musk’s plans for the company.

“Free speech is essential to a functioning democracy. Do you believe Twitter rigorously adheres to this principle?” he asked.

At another point, Mr. Musk wondered, “Is a new platform needed?”

After Mr. Musk inked the deal to buy Twitter last month, he reiterated his free speech stance and said he would take the company private to improve the service. He added that he hoped to increase trust by making Twitter’s technology more transparent, defeating the bots that spam people on the platform and “authenticating all humans.” He also said he hoped his worst critics would remain on Twitter, because “that is what free speech means.”

On Tuesday, he became more specific. “Permanent bans should be extremely rare,” Mr. Musk said, adding that they should be reserved “for accounts that are bots or spam” and “where there’s just no legitimacy to the account at all.”

But he also said that “doesn’t mean that somebody gets to say whatever they want to say.” Mr. Musk said he was in favor of temporary suspensions of accounts “if they say something that is illegal or otherwise just, you know, destructive to the world.” He also raised the idea that a particular tweet could be “made invisible or have very limited traction.”

Apart from Mr. Trump, others who have been indefinitely barred from Twitter for violating its policies include Representative Marjorie Taylor Greene, Republican of Georgia, the far-right figure Milo Yiannopoulos, and celebrities like Tila Tequila. Twitter also labels tweets that are factually inaccurate or that might incite violence.

Inside Twitter on Tuesday, some employees worried that Mr. Musk’s changes would unwind years of work on the company’s policies and unravel millions of dollars of investment in content moderation to stem abuse on the platform, four current and former employees said. Some said they hoped Mr. Musk would lose interest in the site, while others have begun reaching out to recruiters and friends at other tech companies for new opportunities.

Still others were excited at the prospect of Mr. Musk in charge, the current and former employees said. Mr. Musk has pitched investors on quintupling Twitter’s revenue and on the service topping more than 900 million users by 2028, up from 217 million or so today.
"""

In [150]:
%%time
text_summary = get_summary(tokenizer_large, model_large, test_text)
print(text_summary)

Ignored unknown kwarg option direction
Input X tokenized. Generating Summary ...
Summary Generated. Decoding Summary ...
Summary Decoded.
['Elon Musk said on Tuesday that he would “reverse the permanent ban” of former President Donald J. Trump on Twitter and let him back on the social network, in one of the first specific comments by Mr. Musk, who struck a deal last month to buy Twitter for $44 billion, said at a Financial Times conference that the company’s decision to bar Mr. The billionaire, who also leads the electric carmaker Tesla and the rocket company SpaceX, has called himself a “free speech absolutist” and has said he is unhappy with how Twitter decides what can and cannot be posted online.']
CPU times: user 6min 8s, sys: 6min 46s, total: 12min 55s
Wall time: 2min 21s


In [151]:
%%time
text_headline = get_summary(tokenizer_large, model_finetune, text_summary)
print(text_headline)

Ignored unknown kwarg option direction
Input X tokenized. Generating Summary ...
Summary Generated. Decoding Summary ...
Summary Decoded.
['Elon Musk to let Trump back on Twitter']
CPU times: user 49.1 s, sys: 53.5 s, total: 1min 42s
Wall time: 21.7 s
